In [ ]:
!pip install -q --upgrade torch torchvision torchaudio
!pip install -q --upgrade transformers datasets
!pip install -q --upgrade torchmetrics pycocotools

In [ ]:
from transformers import AutoImageProcessor, AutoModelForObjectDetection
import torch
from PIL import Image
import requests

id2label = {
    0: "Infiltration",
    1: "Atelectasis",
    2: "Cardiomegaly",
    3: "Pleural effusion",
    4: "Pneumothorax",
    5: "Nodule/Mass",
}

label2id = {v: k for k, v in id2label.items()}

In [ ]:
from transformers import DetrForObjectDetection, DetrImageProcessor
import torch

model = DetrForObjectDetection.from_pretrained(
    "facebook/detr-resnet-50",
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True,
)
processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.59k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/detr-resnet-50 were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DetrForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DetrForObjectDetection were not initialized from the model checkpoin

preprocessor_config.json:   0%|          | 0.00/290 [00:00<?, ?B/s]

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

DetrForObjectDetection(
  (model): DetrModel(
    (backbone): DetrConvModel(
      (conv_encoder): DetrConvEncoder(
        (model): FeatureListNet(
          (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
          (bn1): DetrFrozenBatchNorm2d()
          (act1): ReLU(inplace=True)
          (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
          (layer1): Sequential(
            (0): Bottleneck(
              (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (bn1): DetrFrozenBatchNorm2d()
              (act1): ReLU(inplace=True)
              (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (bn2): DetrFrozenBatchNorm2d()
              (drop_block): Identity()
              (act2): ReLU(inplace=True)
              (aa): Identity()
              (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      

In [ ]:
!kaggle datasets download zfturbo/nih-chest-xray-dataset-bbox-for-vinbigdata
!unzip nih-chest-xray-dataset-bbox-for-vinbigdata.zip -d data/

Dataset URL: https://www.kaggle.com/datasets/zfturbo/nih-chest-xray-dataset-bbox-for-vinbigdata
License(s): unknown
100% 304M/304M [00:16<00:00, 21.9MB/s]
100% 304M/304M [00:16<00:00, 19.5MB/s]
Archive:  nih-chest-xray-dataset-bbox-for-vinbigdata.zip
  inflating: data/nih.csv            
  inflating: data/nih/00000032_037.png  
  inflating: data/nih/00000072_000.png  
  inflating: data/nih/00000147_001.png  
  inflating: data/nih/00000149_006.png  
  inflating: data/nih/00000181_061.png  
  inflating: data/nih/00000211_010.png  
  inflating: data/nih/00000211_016.png  
  inflating: data/nih/00000211_019.png  
  inflating: data/nih/00000211_041.png  
  inflating: data/nih/00000344_003.png  
  inflating: data/nih/00000377_004.png  
  inflating: data/nih/00000398_003.png  
  inflating: data/nih/00000457_004.png  
  inflating: data/nih/00000468_017.png  
  inflating: data/nih/00000468_033.png  
  inflating: data/nih/00000468_041.png  
  inflating: data/nih/00000583_008.png  
  inflating: d

In [ ]:
import os
import json
import pandas as pd

def nih_csv_to_coco(csv_path, image_dir, output_json):

  df = pd.read_csv(csv_path)

  images = []
  annotations = []
  categories = []
  category_id = 0  # Kategoriler için başlangıç ID'si
  annotation_id = 0 # Açıklamalar için başlangıç ID'si

  for index, row in df.iterrows():
    image_id = row['image_id']

    image_name = f"{image_id}.png" 

    image_id = int(image_id.replace("_", ""))

    images.append({
        "id": image_id,
        "file_name": image_name,
        "width": 1024,  #row['width'],
        "height": 1024, #row['height'],
    })

    category_name = row['class_name']
    if category_name not in [c["name"] for c in categories]:
      categories.append({
          "id": category_id,
          "name": category_name,
          "supercategory": "nih",  # Genel bir üst kategori kullanıyoruz
      })
      category_id += 1

    category_id_for_obj = next(c["id"] for c in categories if c["name"] == category_name)

    annotations.append({
        "id": annotation_id,
        "image_id": image_id,
        "category_id": category_id_for_obj,
        "segmentation": [],
        "bbox": [row['x_min'], row['y_min'], row['x_max'] - row['x_min'], row['y_max'] - row['y_min']],  # COCO formatında [x, y, width, height]
        "iscrowd": 0,  # Kalabalık nesneler için 1, aksi takdirde 0
        "area": (row['x_max'] - row['x_min']) * (row['y_max'] - row['y_min']),  # Sınırlayıcı kutunun alanı
    })
    annotation_id += 1

  coco_data = {
      "info": {
          "description": "NIH Chest X-ray Dataset",
          "url": None,
          "version": "1.0",
          "year": 2023,
          "contributor": "User",
      },
      "images": images,
      "annotations": annotations,
      "categories": categories,
  }

  with open(output_json, "w") as f:
    json.dump(coco_data, f)

  print(f"COCO verileri {output_json} dosyasına kaydedildi.")

In [ ]:
csv_path = "/content/data/nih.csv"  # NIH CSV dosyasının yolu
image_dir = "/content/data/nih"  # Resimlerin bulunduğu dizin
output_json = "nih_coco.json"  # COCO verilerinin kaydedileceği JSON dosyası

nih_csv_to_coco(csv_path, image_dir, output_json)

COCO verileri nih_coco.json dosyasına kaydedildi.


In [ ]:
from pycocotools.coco import COCO

coco = COCO("nih_coco.json")

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [ ]:
from transformers import Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader
from pycocotools.coco import COCO
import torch
import os
from PIL import Image
import numpy as np
import albumentations as A
import cv2
from torch.utils.data import random_split

class NIHDataset(Dataset):
    def __init__(self, annotation_file, image_dir, processor):
        self.coco = COCO(annotation_file)
        self.image_dir = image_dir
        self.processor = processor
        self.image_ids = self.coco.getImgIds()

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx):
        image_id = self.image_ids[idx]
        image_info = self.coco.loadImgs(image_id)
        image_path = os.path.join(self.image_dir, image_info[0]['file_name'])
        image = Image.open(image_path).convert('RGB')
        image = np.array(image)[:, :, ::-1]
        annotations = self.coco.loadAnns(self.coco.getAnnIds(image_id))

        targets = {
            'image_id': torch.tensor([image_id]),
            'annotations': annotations,
        }

        inputs = self.processor(images=image, annotations=targets, return_tensors="pt").to(device)

        return inputs


/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.22 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [ ]:
dataset = NIHDataset(annotation_file="nih_coco.json", image_dir="/content/data/nih", processor=processor)

train_size = int(0.9 * len(dataset))  # Eğitim kümesinin boyutu
val_size = len(dataset) - train_size  # Doğrulama kümesinin boyutu

train_dataset, val_dataset = random_split(dataset, [train_size, val_size], generator=torch.Generator().manual_seed(42))

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [ ]:
def collate_fn(batch):
    pixel_values = [item["pixel_values"].squeeze(0).cpu() for item in batch]
    encoding = processor.pad(pixel_values, return_tensors="pt")
    labels = [item["labels"][0] for item in batch]

    batch = {}
    batch["pixel_values"] = encoding["pixel_values"].to(device)
    batch["pixel_mask"] = encoding["pixel_mask"].to(device)
    batch["labels"] = labels

    return batch

In [ ]:
from transformers.image_transforms import center_to_corners_format

def convert_bbox_yolo_to_pascal(boxes, image_size):
    boxes = center_to_corners_format(boxes)

    height, width = image_size
    boxes = boxes * torch.tensor([[width, height, width, height]])

    return boxes

In [ ]:
import torch
from typing import Mapping, Optional
from torchmetrics.detection.mean_ap import MeanAveragePrecision
from transformers.modeling_outputs import ModelOutput
from transformers.trainer_utils import EvalPrediction
from dataclasses import dataclass
import torch
from functools import partial

@dataclass
class ModelOutput:
    logits: torch.Tensor
    pred_boxes: torch.Tensor


@torch.no_grad()
def compute_metrics(
    evaluation_results: EvalPrediction,
    image_processor: AutoImageProcessor,
    threshold: float = 0.0,
    id2label: Optional[Mapping[int, str]] = None,
) -> Mapping[str, float]:


    predictions, targets = evaluation_results.predictions, evaluation_results.label_ids

    image_sizes = []
    post_processed_targets = []
    post_processed_predictions = []
    for batch in targets:
        batch_image_sizes = torch.tensor([x["orig_size"] for x in batch])
        image_sizes.append(batch_image_sizes)
        for image_target in batch:
            boxes = torch.tensor(image_target["boxes"])
            boxes = convert_bbox_yolo_to_pascal(boxes, image_target["orig_size"])
            labels = torch.tensor(image_target["class_labels"])
            post_processed_targets.append({"boxes": boxes, "labels": labels})
    for batch, target_sizes in zip(predictions, image_sizes):
        batch_logits, batch_boxes = batch[1], batch[2]
        output = ModelOutput(logits=torch.tensor(batch_logits), pred_boxes=torch.tensor(batch_boxes))
        post_processed_output = image_processor.post_process_object_detection(
            output, threshold=threshold, target_sizes=target_sizes
        )
        post_processed_predictions.extend(post_processed_output)

    metric = MeanAveragePrecision(box_format="xyxy", class_metrics=True)
    metric.update(post_processed_predictions, post_processed_targets)
    metrics = metric.compute()

    classes = metrics.pop("classes")
    map_per_class = metrics.pop("map_per_class")
    mar_100_per_class = metrics.pop("mar_100_per_class")
    for class_id, class_map, class_mar in zip(classes, map_per_class, mar_100_per_class):
        class_name = id2label[class_id.item()] if id2label is not None else class_id.item()
        metrics[f"map_{class_name}"] = class_map
        metrics[f"mar_100_{class_name}"] = class_mar

    metrics = {k: round(v.item(), 4) for k, v in metrics.items()}

    return metrics

eval_compute_metrics_fn = partial(
    compute_metrics, image_processor=processor, id2label=id2label, threshold=0.0
)

In [ ]:
from transformers import get_cosine_schedule_with_warmup

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=50,
    lr_scheduler_type="cosine",
    fp16=False,
    save_strategy="epoch",
    logging_strategy="epoch",
    max_grad_norm=0.01,
    learning_rate=2e-5,
    weight_decay=1e-4,
    save_total_limit=2,
    metric_for_best_model="eval_map",
    greater_is_better=True,
    load_best_model_at_end=True,
    remove_unused_columns=False,
    evaluation_strategy="epoch",
    report_to="none",
    dataloader_pin_memory=False,
    eval_do_concat_batches=False
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=collate_fn,
    tokenizer=processor,
    compute_metrics=eval_compute_metrics_fn,
)

<ipython-input-37-a501f74edacd>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.evaluate()

{'eval_loss': 2.413383722305298,
 'eval_model_preparation_time': 0.0136,
 'eval_map': 0.003,
 'eval_map_50': 0.0085,
 'eval_map_75': 0.0006,
 'eval_map_small': 0.0,
 'eval_map_medium': 0.05,
 'eval_map_large': 0.0031,
 'eval_mar_1': 0.0632,
 'eval_mar_10': 0.1037,
 'eval_mar_100': 0.1259,
 'eval_mar_small': 0.0,
 'eval_mar_medium': 0.05,
 'eval_mar_large': 0.1266,
 'eval_map_Infiltration': 0.0,
 'eval_mar_100_Infiltration': 0.0,
 'eval_map_Atelectasis': 0.0,
 'eval_mar_100_Atelectasis': 0.0,
 'eval_map_Cardiomegaly': 0.0128,
 'eval_mar_100_Cardiomegaly': 0.55,
 'eval_map_Pleural effusion': 0.0004,
 'eval_mar_100_Pleural effusion': 0.1429,
 'eval_map_Pneumothorax': 0.005,
 'eval_mar_100_Pneumothorax': 0.0625,
 'eval_map_Nodule/Mass': 0.0,
 'eval_mar_100_Nodule/Mass': 0.0,
 'eval_runtime': 11.4329,
 'eval_samples_per_second': 6.91,
 'eval_steps_per_second': 6.91}

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Model Preparation Time,Map,Map 50,Map 75,Map Small,Map Medium,Map Large,Mar 1,Mar 10,Mar 100,Mar Small,Mar Medium,Mar Large,Map Infiltration,Mar 100 Infiltration,Map Atelectasis,Mar 100 Atelectasis,Map Cardiomegaly,Mar 100 Cardiomegaly,Map Pleural effusion,Mar 100 Pleural effusion,Map Pneumothorax,Mar 100 Pneumothorax,Map Nodule/mass,Mar 100 Nodule/mass
1,2.380600,2.479632,0.013600,0.001800,0.005400,0.000900,0.000000,0.000000,0.001800,0.047200,0.109300,0.114100,0.000000,0.000000,0.117100,0.000000,0.000000,0.000300,0.039100,0.009600,0.516700,0.000700,0.128600,0.000000,0.000000,0.000000,0.000000
2,2.307000,2.473914,0.013600,0.001600,0.005300,0.000800,0.000000,0.000000,0.001700,0.037500,0.110500,0.138300,0.000000,0.000000,0.142100,0.000600,0.060000,0.000100,0.030400,0.007400,0.550000,0.000400,0.164300,0.001400,0.025000,0.000000,0.000000
3,2.328100,2.405124,0.013600,0.004900,0.009300,0.004000,0.000000,0.003200,0.004900,0.052800,0.102000,0.132600,0.000000,0.012500,0.135200,0.000000,0.000000,0.000100,0.047800,0.028200,0.533300,0.000900,0.214300,0.000000,0.000000,0.000000,0.000000
4,2.248400,2.305188,0.013600,0.005100,0.011600,0.003900,0.000000,0.000000,0.005100,0.071500,0.159300,0.166300,0.000000,0.001400,0.173400,0.000100,0.010000,0.000900,0.100000,0.027500,0.633300,0.001000,0.150000,0.001000,0.100000,0.000000,0.004500
5,2.200100,2.313330,0.013600,0.002300,0.008300,0.001100,0.000000,0.001400,0.002400,0.040200,0.136500,0.143800,0.000000,0.020100,0.167000,0.000700,0.060000,0.000900,0.082600,0.010500,0.433300,0.001200,0.164300,0.000500,0.100000,0.000000,0.022700
6,2.114300,2.327216,0.013600,0.010900,0.027100,0.005700,0.000000,0.000000,0.010900,0.065300,0.153300,0.158400,0.000000,0.001400,0.173600,0.000000,0.000000,0.000400,0.043500,0.062700,0.616700,0.001400,0.164300,0.000700,0.112500,0.000000,0.013600
7,2.106700,2.299630,0.013600,0.005400,0.014700,0.003900,0.000000,0.000000,0.005500,0.091900,0.144000,0.149300,0.000000,0.014600,0.148700,0.000900,0.050000,0.003100,0.056500,0.026200,0.583300,0.001600,0.128600,0.000600,0.050000,0.000000,0.027300
8,2.101600,2.126888,0.013600,0.005100,0.013900,0.002500,0.000000,0.000000,0.005500,0.083200,0.177000,0.178500,0.000000,0.002800,0.198100,0.000200,0.030000,0.002300,0.073900,0.024500,0.700000,0.001500,0.157100,0.001900,0.087500,0.000300,0.022700
9,2.008400,2.090580,0.013600,0.009100,0.019900,0.007400,0.000000,0.000000,0.009200,0.083300,0.180800,0.191400,0.000000,0.006300,0.205200,0.000100,0.010000,0.000400,0.073900,0.048000,0.633300,0.003900,0.264300,0.002100,0.162500,0.000000,0.004500
10,2.014800,2.160003,0.013600,0.007000,0.014900,0.002700,0.000000,0.000200,0.007200,0.105700,0.190200,0.213000,0.000000,0.013900,0.223300,0.000700,0.040000,0.001100,0.152200,0.032700,0.683300,0.001900,0.185700,0.005900,0.212500,0.000000,0.004500


TrainOutput(global_step=35200, training_loss=1.7593450424887918, metrics={'train_runtime': 9077.7009, 'train_samples_per_second': 3.878, 'train_steps_per_second': 3.878, 'total_flos': 1.6819220385792e+19, 'train_loss': 1.7593450424887918, 'epoch': 50.0})

In [ ]:
trainer.save_model("./detr_nih")

In [ ]:
!zip -r detr_nih.zip detr_nih/

  adding: detr_nih/ (stored 0%)
  adding: detr_nih/preprocessor_config.json (deflated 48%)
  adding: detr_nih/config.json (deflated 61%)
  adding: detr_nih/model.safetensors (deflated 7%)
  adding: detr_nih/training_args.bin (deflated 51%)


In [ ]:
from google.colab import files
files.download('/content/detr_nih.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>